In [ ]:
%pip install numpy
%pip install pandas

In [1]:
import numpy as np
import pandas as pd

from ahc import AHC
from kmeans import K_Means

%load_ext autoreload
%autoreload 2

In [2]:
data = pd.read_csv("iris.csv")

X = data.iloc[:, :4]
# y = data.iloc[:, 4]

In [3]:
# k = int(input("Nro de clusters (k): "))
k = 3

In [4]:
# TODO: mover para silhouette.py e importar aqui
def my_silhouette_score(X: np.ndarray, labels: np.array):
    # number of samples
    n = X.shape[0]

    # initialize silhouette scores
    sil_scores = np.zeros(n)

    # calculate pairwise distances manually
    D = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            D[i, j] = np.sqrt(np.sum((X[i] - X[j]) ** 2))

    for i in range(n):
        # Find the cluster of the current sample
        own_cluster = labels[i]

        # Mask for the own cluster
        mask_own = labels == own_cluster

        # a(i): the average distance to all other points in the same cluster - intra cluster - compactness
        if np.sum(mask_own) > 1:
            a_i = np.sum(D[i, mask_own]) / (np.sum(mask_own) - 1)
        else:
            a_i = 0

        # b(i): the minimum average distance to points in any other cluster - inter cluster - separation
        b_i = np.inf
        for label in np.unique(labels):
            if label == own_cluster:
                continue
            mask_other = labels == label
            b_i = min(b_i, np.mean(D[i, mask_other]))

        # Compute the silhouette score for the current sample
        sil_scores[i] = (b_i - a_i) / max(a_i, b_i)

    return np.mean(sil_scores)

In [5]:
def my_silhouette_score_2(X, labels):
    n = X.shape[0]
    unique_labels = np.unique(labels)
    silhouette_vals = np.zeros(n)

    dist_matrix = np.sqrt((X[:, np.newaxis] - X[np.newaxis, :]) ** 2).sum(axis=2)

    for i in range(n):
        same_cluster_mask = labels == labels[i]
        a_i = np.mean(dist_matrix[i, same_cluster_mask])
        b_i = np.inf


        for label in unique_labels:
            if label == labels[i]:
                continue
            other_cluster_mask = labels == label
            b_i = min(b_i, np.mean(dist_matrix[i, other_cluster_mask]))

        silhouette_vals[i] = (b_i - a_i) / max(a_i, b_i)
    
    silhouette_avg = np.mean(silhouette_vals)
    return silhouette_avg

In [6]:
print("K-Means")
print("k: ", k)
model = K_Means(k=k)
labels, centroids = model.fit_predict(X.to_numpy())
model.print_centroids()
print("Labels: ")
print(labels)
print("Silhouette Score: ", my_silhouette_score(X.to_numpy(), labels))
print("Silhouette Score 2: ", my_silhouette_score_2(X.to_numpy(), labels))
from sklearn.metrics import silhouette_score
print("SKLEARN Silhouette Score:", silhouette_score(X.to_numpy(), labels))

K-Means
k:  3
Centroid 0: [6.2        2.85714286 4.9        1.68367347]
Centroid 1: [7.0962963  3.11481481 5.7        1.9962963 ]
Centroid 2: [5.15       3.16216216 2.29459459 0.58648649]
Labels: 
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 2 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 2 0 1 0 1 0 1 1 0 1 1 1 0
 0 1 0 0 0 1 1 1 0 1 0 1 0 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 1 1 1 0 0
 0 0]
Silhouette Score:  0.5237701851863256
Silhouette Score 2:  0.5386966351633725
SKLEARN Silhouette Score: 0.5237701851863256


In [ ]:
print("AHC")
ahc = AHC(X, save_output=False)
ahc.run()
# TODO: como calcular os centroids para aplicar silhueta simplificada